In [2]:
import pandas as pd
from datetime import datetime
import time
from time import gmtime, strftime
import os
import ast
import uuid

# Web scraping
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

from instascrape import *

In [24]:
# Obtain list of posts per profile
df1 = pd.read_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv',
                  index_col = False, encoding = 'ISO-8859-1')
df1 = df1[(df1.ExtrType == 'ProfileID-Instagram') & (df1.Output1.str.contains('PostsDetails'))]
df1 = df1.sort_values(by = ['Key1', 'ExtractionDateTime'], ascending = False).reset_index(drop = True)
df1 = df1.drop_duplicates(subset = ['Key1']).reset_index(drop = True)

In [35]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_PrepareOutputIntoDF.ipynb

In [ ]:
os. chdir('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/PythonScripts')
%run ./DataProcessing_TextPreparation.ipynb

In [36]:
# Initialize web scraping
PATH = r"C:\Users\Esteban Guerrero\Downloads\chromedriver_win32\chromedriver.exe"
chOpt = webdriver.ChromeOptions()
chOpt.add_argument("--lang=en")

# Initialize Instagram web scraping
driverIns = webdriver.Chrome(PATH, options = chOpt)
driverIns.get("https://www.instagram.com/")
driverIns.maximize_window()

# login
time.sleep(5)
usernameIns = driverIns.find_element_by_css_selector("input[name='username']")
passwordIns = driverIns.find_element_by_css_selector("input[name='password']")
usernameIns.clear()
passwordIns.clear()
usernameIns.send_keys("uxyz")
passwordIns.send_keys("pxyz")
loginIns = driverIns.find_element_by_css_selector("button[type='submit']").click()

# Save your login info?
time.sleep(10)
notnowIns = driverIns.find_element_by_xpath("//button[contains(text(), 'Not Now')]").click()
##turn off notifaction
time.sleep(10)
notnow2Ins = driverIns.find_element_by_xpath("//button[contains(text(), 'Not Now')]").click()

In [37]:
# Function to extract data per user post and extract some comments

def INSPostWebScrap(INSProfile1, INSAlias1, INSPostList1):
    
    postOutputDF = pd.DataFrame()
    commentsOutputDF = pd.DataFrame()
    
    for pst in INSPostList1:
            
        # Extract post data per post

        pst1 = Post(pst)
        pst1.scrape(webdriver = driverIns)
        pst1dict = pst1.to_dict()

        postOutput = {'Output1': {'User': INSAlias1, 'Post': {'id': str(pst1dict.get('id')),
                                                                'shortcode': str(pst1dict.get('shortcode')),
                                                                'tagged_users': TextCleansing(str(pst1dict.get('tagged_users'))), 
                                                                'comments': str(pst1dict.get('comments')),
                                                                'upload_date': str(pst1dict.get('upload_date')),
                                                                'likes': str(pst1dict.get('likes')),
                                                                'location': str(pst1dict.get('location')),
                                                                'video_view_count': str(pst1dict.get('video_view_count')),
                                                                'hashtags': TextCleansing(str(pst1dict.get('hashtags'))),
                                                                'caption': TextCleansing(pst1dict.get('caption'))}},
                      'Key1': INSProfile1, 'Key2': str(pst1dict.get('shortcode'))}

        postOutputDF = postOutputDF.append(postOutput, ignore_index=True)

        # Click the load more comments button 5 times

        check1 = True
        m = 1
        while ((check1) and (m <= 5)):
            try:
                moreCommentsButton = driverIns.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/li/div/button/span').click()
                time.sleep(2)
            except:
                check1 = False

            m = m + 1

        # Extract comments data

        check2 = True
        n = 1
        while check2:
            try:
                authorXPath = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/ul['+str(n)+']/div/li/div/div[1]/div[2]/h3/div/span/a'
                commentXPath = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/ul['+str(n)+']/div/li/div/div[1]/div[2]/span'
                likesXPath = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/ul['+str(n)+']/div/li/div/div[1]/div[2]/div/div/button[1]'
                dateXPath = '//*[@id="react-root"]/section/main/div/div[1]/article/div[3]/div[1]/ul/ul['+str(n)+']/div/li/div/div[1]/div[2]/div/div/a/time'

                authorText = driverIns.find_element_by_xpath(authorXPath).text
                likesText = driverIns.find_element_by_xpath(likesXPath).text
                dateText = driverIns.find_element_by_xpath(dateXPath).get_attribute('datetime')
                commentText = driverIns.find_element_by_xpath(commentXPath).text

                commentOutput = {'Output1': {'User': INSAlias1, 'CommentLine': {'Author': TextCleansing(authorText),
                                                                                'Likes': likesText,
                                                                                'Date': dateText,
                                                                                'id_str': str(uuid.uuid4()),
                                                                                'Comment': TextCleansing(commentText)}},
                                 'Key1': INSProfile1, 'Key2': pst1dict.get('shortcode')}                                  

                commentsOutputDF = commentsOutputDF.append(commentOutput, ignore_index=True)
                n = n + 1
            except:
                check2 = False

        time.sleep(5)
    
    return postOutputDF, commentsOutputDF

In [38]:
# Run loop to extract posts data and comments

INSPostDF = pd.DataFrame()
INSCommDF = pd.DataFrame()

for i in range(0, len(df1.Output1)):
    print('***', ast.literal_eval(df1.iat[i, 2]).get('PostsDetails').get('Alias'), '\nstart\t', datetime.now())
    INSpdf, INScdf = INSPostWebScrap(ast.literal_eval(df1.iat[i, 2]).get('PostsDetails').get('Profile'),
                                     ast.literal_eval(df1.iat[i, 2]).get('PostsDetails').get('Alias'),
                                     ast.literal_eval(df1.iat[i, 2]).get('PostsDetails').get('PostsList')[:5])
    INSPostDF = INSPostDF.append(INSpdf, ignore_index=True)
    INSCommDF = INSCommDF.append(INScdf, ignore_index=True)
    print('end\t', datetime.now())

*** PLibCABA 
start	 2021-08-02 12:45:35.817523
end	 2021-08-02 12:46:32.579392
*** Espert 
start	 2021-08-02 12:46:32.580385
end	 2021-08-02 12:49:53.492342
*** Milei 
start	 2021-08-02 12:49:53.496341
end	 2021-08-02 12:53:49.167504
*** ClauG 
start	 2021-08-02 12:53:49.168503
end	 2021-08-02 12:54:35.756409


In [39]:
InsPostDetDF = PrepareOutputDF('K12', INSPostDF, 'Automatic', 'PostDetails-Instagram')
InsCommtDF = PrepareOutputDF('K12', INSCommDF, 'Automatic', 'PostComments-Instagram')

In [40]:
DF = pd.read_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index_col = False, encoding = 'ISO-8859-1')
DF = DF.append([InsPostDetDF, InsCommtDF], ignore_index = True)

DF.to_csv('C:/Users/Esteban Guerrero/SocialNetworkProject/Iteration2/Datasets/NetworkFeedRawOutput.csv', index = False)

In [41]:
driverIns.quit()